In [1]:
import numpy as np
import copy
from scipy.linalg import eigh, svd
from scipy.sparse.linalg import expm, expm_multiply
from scipy.sparse import csc_matrix, csr_matrix, kron, lil_matrix, eye
import datetime
import os
import glob

In [2]:
num_qubits = 8
run_davidson = 0
M = 10
tau = 0.1

In [3]:


x_hopping = 1
y_hopping = 1
z_hopping = 1
model = "xxx"
use_trotter = 0

# full_path = os.path.realpath(__file__)
# path, filename = os.path.split(full_path)

pauli = np.array([np.array([[1,0],[0,1]]), np.array([[0,1],[1,0]]), np.array([[0,-1.j],[1.j,0]]), np.array([[1,0],[0,-1]])])
pauli_tensor = np.array([[np.kron(pauli[i], pauli[j]) for i in range(4)] for j in range(4)])

# building operators
def kronecker_pad(paulis, num_qubits, sites):
    ''' pads a 1- or 2- local operator with identities on other sites to get 2^n by 2^n matrix '''
    kron_list = [np.eye(2) for i in range(num_qubits)]
    for i, site in enumerate(sites):
        kron_list[site] = paulis[i]

    padded_matrix = kron_list[0]
    for i in range(1, len(kron_list)):
        padded_matrix = np.kron(kron_list[i], padded_matrix)
    return padded_matrix

def heisenberg(num_qubits, bias_coeff=1.0, x_hopping_coeff=1.0, y_hopping_coeff=1.0, z_hopping_coeff=1.0):
    terms = []
    for i in range(num_qubits):
        bias = bias_coeff*kronecker_pad([pauli[3]], num_qubits, [i])
        terms.append(bias)

    for i in range(num_qubits-1):
        z_hop = z_hopping_coeff*kronecker_pad([pauli[3], pauli[3]], num_qubits, [i, (i+1)%num_qubits])
        terms.append(z_hop)
        y_hop = y_hopping_coeff*kronecker_pad([pauli[2], pauli[2]], num_qubits, [i, (i+1)%num_qubits])
        terms.append(y_hop)
        x_hop = x_hopping_coeff*kronecker_pad([pauli[1], pauli[1]], num_qubits, [i, (i+1)%num_qubits])
        terms.append(x_hop)
    return sum(terms)

def trotter(num_qubits, bias_coeff=1.0, x_hopping_coeff=1.0, y_hopping_coeff=1.0, z_hopping_coeff=1.0):
    even_terms = []
    odd_terms = []

    for i in range(0, num_qubits-1, 2):
        z_hop = z_hopping_coeff*kronecker_pad([pauli[3], pauli[3]], num_qubits, [i, (i+1)%num_qubits])
        odd_terms.append(z_hop)
        y_hop = y_hopping_coeff*kronecker_pad([pauli[2], pauli[2]], num_qubits, [i, (i+1)%num_qubits])
        odd_terms.append(y_hop)
        x_hop = x_hopping_coeff*kronecker_pad([pauli[1], pauli[1]], num_qubits, [i, (i+1)%num_qubits])
        odd_terms.append(x_hop)
    for i in range(1, num_qubits-1, 2):
        z_hop = z_hopping_coeff*kronecker_pad([pauli[3], pauli[3]], num_qubits, [i, (i+1)%num_qubits])
        even_terms.append(z_hop)
        y_hop = y_hopping_coeff*kronecker_pad([pauli[2], pauli[2]], num_qubits, [i, (i+1)%num_qubits])
        even_terms.append(y_hop)
        x_hop = x_hopping_coeff*kronecker_pad([pauli[1], pauli[1]], num_qubits, [i, (i+1)%num_qubits])
        even_terms.append(x_hop)

    even = sum(even_terms)
    odd = sum(odd_terms)

    return even, odd

def fast_heisenberg(L):
    def FlipFlop(n, i, j):
        v = list(format(n, '0{}b'.format(L)))
        if (v[i] != '0' and v[j] != '1'):
            v[i] = '0'
            v[j] = '1'
            return int(''.join(v), 2)
        else:
            return -1

    sprs = lil_matrix((2**L, 2**L), dtype=np.int8)
    for i in range(L-1):
        for j in range(2**L):
            h = FlipFlop(j, i, i+1)
            if (h != -1):
                sprs[j, h] = 2
                sprs[h, j] = 2

            v = lambda k: 1-2*int(format(j, '0{}b'.format(L))[k])
            sprs[j, j] += v(i) * v(i+1)

    # bias term
    for i in range(L):
        for j in range(2**L):
            sprs[j, j] += v(i)

    return sprs.tocsc()

def correction_state(ham, energy, state, tau=0.0001):
    #op = expm(-tau*(ham-energy))
    #correction_state = op @ state
    correction_state = ham @ state - energy*state
    return correction_state / np.linalg.norm(correction_state)

def eff_ham(ham, basis_set):
    eff_H = np.eye(len(basis_set), dtype=complex)
    for i in range(len(basis_set)):
        for j in range(i,len(basis_set)):
            overlap = basis_set[i].conj().transpose(copy=False).dot(ham.dot(basis_set[j]))[0,0]
            eff_H[i][j] = overlap
            if (i != j):
                eff_H[j][i] = np.conj(overlap)
    return eff_H

def eff_overlap(basis_set):
    eff_S = np.eye(len(basis_set), dtype=complex)
    for i in range(len(basis_set)):
        for j in range(i,len(basis_set)):
            overlap = basis_set[i].conj().transpose(copy=False).dot(basis_set[j])[0,0]
            eff_S[i][j] = overlap
            if (i != j):
                eff_S[j][i] = np.conj(overlap)
    return eff_S

def qdavidson_iter(ham, basis_set, tol=0.5):
    num_basis = len(basis_set)
    eff_H = eff_ham(ham, basis_set)
    eff_S = eff_overlap(basis_set)
    evals, evecs = eigh(eff_H, eff_S)
    estates = [np.array(sum([evecs[:,i][j] * basis_set[j] for j in range(num_basis)])) for i in range(num_basis)]
    new_basis_set = copy.deepcopy(basis_set)
    residue_vals = []
    for i in range(num_basis):
        val = np.linalg.norm((ham @ estates[i]) - (evals[i] * estates[i]))
        residue_vals.append(val)
        if val > tol:
            state = correction_state(ham, evals[i], estates[i])
            if linear_independence(state, new_basis_set, eff_S, tol):
                '''
                eff_S = np.pad(eff_S, ((0, 1), (0, 1)), mode='constant')
                for i in range(len(new_basis_set)):
                    overlap = state.conj().T @ new_basis_set[i]
                    eff_S[i][len(new_basis_set)] = overlap
                    eff_S[len(new_basis_set)][i] = overlap
                '''
                new_basis_set.append(state)
                eff_S = eff_overlap(new_basis_set)
                eff_H = eff_ham(ham, new_basis_set)

    return evals, estates, residue_vals, new_basis_set, eff_H, eff_S

def linear_independence(correction_vec, basis_set, eff_S, tol=0.01):
    b = np.array([correction_vec.conj().T.dot(basis_set[i]) for i in range(len(basis_set))])
    if np.all(np.round(eff_S, 8) == np.eye(len(basis_set))):
        return np.linalg.norm(b) < tol
    else:
        return np.linalg.norm(np.linalg.pinv(eff_S).dot(b)) < tol

def qdavidson(ham, initial_basis_set, num_iter, tol=0.5):
    basis_set = copy.deepcopy(initial_basis_set)
    for i in range(num_iter):
        evals, estates, residue_vals, basis_set, eff_H, eff_S = qdavidson_iter(ham, basis_set, tol)
    return evals, estates, residue_vals, basis_set, eff_H, eff_S

def QDFFEvolve(S_inv, H, t, basis, init):
    ct = expm(-1j * t * S_inv @ H) @ init
    return sum([ct[i] * basis[i] for i in range(len(ct))])


# --------------------------------------------------------

epsilon = 0.1  # we want the fidelity with the exact state to be at least this at time = tf

tf = 10
ts = np.linspace(0, tf, 200)

eff_S_eps = 1e-9  # cutoff values in eff_S less than this value

cs = list("01010101010101010101010101010101010101010101010101")



In [15]:
ham = heisenberg(2, x_hopping_coeff=x_hopping, y_hopping_coeff=y_hopping, z_hopping_coeff=z_hopping, bias_coeff=1)
expm_trot = expm(-1j * tau * ham)

In [19]:
np.round(np.linalg.matrix_power(expm_trot, 2), 3)

array([[0.825-0.565j, 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.903+0.183j, 0.077-0.382j, 0.   +0.j   ],
       [0.   +0.j   , 0.077-0.382j, 0.903+0.183j, 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.98 +0.199j]])

In [18]:
np.round(expm_trot**5, 3)

array([[0.071-0.997j, 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.794+0.434j, 0.   -0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   -0.j   , 0.794+0.434j, 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.878+0.479j]])

In [22]:
np.round(expm_trot @ expm_trot, 3)

array([[0.825-0.565j, 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.903+0.183j, 0.077-0.382j, 0.   +0.j   ],
       [0.   +0.j   , 0.077-0.382j, 0.903+0.183j, 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.98 +0.199j]])

In [17]:
np.round(expm_trot**5, 5) == np.round(np.linalg.matrix_power(expm_trot, 5), 5)

array([[ True,  True,  True,  True],
       [ True, False, False,  True],
       [ True, False, False,  True],
       [ True,  True,  True,  True]])

In [24]:
arr = np.eye(2)*2
arr**2

array([[4., 0.],
       [0., 4.]])

In [27]:
np.round(expm_trot, 2)

array([[0.96-0.3j, 0.  +0.j , 0.  +0.j , 0.  +0.j ],
       [0.  +0.j , 0.98+0.1j, 0.02-0.2j, 0.  +0.j ],
       [0.  +0.j , 0.02-0.2j, 0.98+0.1j, 0.  +0.j ],
       [0.  +0.j , 0.  +0.j , 0.  +0.j , 1.  +0.1j]])

In [28]:
arr = np.array([[0,2],[2,0]])
arr**2

array([[0, 4],
       [4, 0]])

In [29]:
arr @ arr

array([[4, 0],
       [0, 4]])